In [2]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_txt= f.read()

print("Total number of character:",len(raw_txt))
print(raw_txt[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [3]:
import re
text = "Hello, world. This, is a test."
result= re.split(r'(\s)',text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [4]:
text = "Hello, world. This, is a test."
result= re.split(r'([,.]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [5]:
result=[item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [6]:
text = "Hello, world. This, is-- a test?"
result= re.split(r'([,.?":;_!\'()]|--|\s)',text)
result=[item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', '--', 'a', 'test', '?']


In [7]:
preprocessed= re.split(r'([,.?":;_!\'()]|--|\s)',raw_txt)
preprocessed=[item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [8]:
print(len(preprocessed))

4690


In [9]:
all_words= sorted(set(preprocessed))
vocab_size= len(all_words)
print(vocab_size)

1130


In [10]:
vocab={token:integer for integer,token in enumerate(all_words)}
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [11]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed= re.split(r'([,.?":;_!\'()]|--|\s)',text)
    preprocessed=[item for item in preprocessed if item.strip()]
    ids= [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text=" ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([.,?!()\'])',r'\1',text)
    return text


In [12]:
tokenizer= SimpleTokenizerV1(vocab)
text= """It's the last he painted, you know,"
Mrs, Gisburn said with pride."""
ids= tokenizer.encode(text)

In [13]:
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 5, 38, 851, 1108, 793, 7]


In [14]:
tokenizer.decode(ids)

'It\' s the last he painted, you know, " Mrs, Gisburn said with pride.'

**Adding** Special Context Tokens

In [15]:
all_tokens= sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab= {token:integer for integer,token in enumerate(all_tokens)}

In [16]:
len(vocab.items())

1132

In [17]:
for i,item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [18]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [19]:
tokenizer= SimpleTokenizerV2(vocab)
text1="Hello, do you like tea?"
text2="In the sunlit terraces of the palace."
text=" <|endoftext|> ".join((text1,text2))

In [20]:
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [21]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [22]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [23]:
#BPE
#HANDLES OOV AND HAS LESS VOCAB SIZE
#SUBWORD TOKENIZATION

In [24]:
!pip3 install tiktoken

In [25]:
import tiktoken
tokenizer= tiktoken.get_encoding("gpt2")
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
integer= tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integer)
string = tokenizer.decode(integer)
print(string)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [26]:
#data sampling with sliding window

In [27]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_txt= f.read()

encoded_txt= tokenizer.encode(raw_txt)
print(len(encoded_txt))

5145


In [28]:
context_size=10
x=encoded_txt[:context_size]
y=encoded_txt[1:context_size+1]
print(x,y)

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138] [367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257]


In [29]:
for i in range(1,context_size+1):
  context= encoded_txt[:i]
  target= encoded_txt[i]
  print(context,target)
  print(tokenizer.decode(context),tokenizer.decode([target]))

[40] 367
I  H
[40, 367] 2885
I H AD
[40, 367, 2885] 1464
I HAD  always
[40, 367, 2885, 1464] 1807
I HAD always  thought
[40, 367, 2885, 1464, 1807] 3619
I HAD always thought  Jack
[40, 367, 2885, 1464, 1807, 3619] 402
I HAD always thought Jack  G
[40, 367, 2885, 1464, 1807, 3619, 402] 271
I HAD always thought Jack G is
[40, 367, 2885, 1464, 1807, 3619, 402, 271] 10899
I HAD always thought Jack Gis burn
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899] 2138
I HAD always thought Jack Gisburn  rather
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138] 257
I HAD always thought Jack Gisburn rather  a


In [30]:
# implement data loader
from torch.utils.data import Dataset,DataLoader
import torch
class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids=[]
    self.target_ids=[]

    token_ids= tokenizer.encode(txt,allowed_special={"<|endoftext|>"})
    for i in range(0,len(token_ids)-max_length,stride):
      input_chunk= token_ids[i:i+max_length]
      target_chunk= token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
      return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx],self.target_ids[idx]

In [31]:
def create_dataloader_v1(txt,batch_Size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
  tokenizer= tiktoken.get_encoding("gpt2")
  dataset= GPTDatasetV1(txt,tokenizer,max_length,stride)
  dataloader= DataLoader(dataset,batch_size=batch_Size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
  return dataloader

In [32]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_txt= f.read()

In [33]:
import torch
dataloader= create_dataloader_v1(raw_txt,batch_Size=4,max_length=20,stride=5,shuffle=True,drop_last=True,num_workers=0)
data_iter= iter(dataloader)
first_batch= next(data_iter)
print(first_batch)

[tensor([[  373,  7067, 29396, 18443, 12271,   290, 45592,    12, 14792,  5986,
           351,   257,  8839,   326,  7284, 35924,   262, 12306,   395,  4133],
        [   40, 27846,   706,   683,    11,  7425,   416,   465,   938,  1573,
            13, 12622, 41379,   293,   373,    11,   287,  1109,    11,  5033],
        [  887,   673,  3521,   470,  6842,   407,   284,   423,   477,   262,
          8263,    12,  9649,   351,   607,    13,  1375,  3521,   470,  6842],
        [  290,  1807,   683, 32081,   290, 44852,    88,    13,  2735,   314,
          2497,   326,   339,   373, 21840,    13,   198,   198,     1,    40]]), tensor([[ 7067, 29396, 18443, 12271,   290, 45592,    12, 14792,  5986,   351,
           257,  8839,   326,  7284, 35924,   262, 12306,   395,  4133,    13],
        [27846,   706,   683,    11,  7425,   416,   465,   938,  1573,    13,
         12622, 41379,   293,   373,    11,   287,  1109,    11,  5033,   262],
        [  673,  3521,   470,  6842,   407,

In [34]:
#token embeddings
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)
print(embedding_layer(input_ids))

Parameter containing:
tensor([[-1.2141, -0.6169,  1.0109],
        [ 0.1939, -0.6726, -0.7582],
        [-0.9343, -0.4632, -0.0573],
        [ 1.4513, -1.3632,  0.2089],
        [ 1.5303,  1.0725, -0.2980],
        [-0.7904, -0.9103, -0.1622]], requires_grad=True)
tensor([[-0.9343, -0.4632, -0.0573],
        [ 1.4513, -1.3632,  0.2089],
        [-0.7904, -0.9103, -0.1622],
        [ 0.1939, -0.6726, -0.7582]], grad_fn=<EmbeddingBackward0>)


In [36]:
#postional embeddings
vocab_size=50257
output_dim=256
max_length=4
dataloader= create_dataloader_v1(raw_txt,batch_Size=8,max_length=max_length,stride=max_length,shuffle=False)
data_iter= iter(dataloader)
inputs , targets= next(data_iter)

In [37]:
token_embedding_layer= torch.nn.Embedding(vocab_size,output_dim)
token_embeddings= token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [39]:
context_length= max_length
pos_embedding_layer= torch.nn.Embedding(context_length,output_dim)
pos_embeddings= pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [40]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
